In [215]:
import requests
from bs4 import BeautifulSoup
import os

In [12]:
url = 'https://github.com/topics/'

In [6]:
response = requests.get(url)

In [7]:
response.status_code

200

In [19]:
page_contents = response.text

In [23]:
with open('webpage.html', 'w',  encoding="utf-8") as f:
    f.write(page_contents)

In [25]:
doc = BeautifulSoup(page_contents, 'html.parser')

In [78]:
import pandas as pd

In [96]:
topic_doc = BeautifulSoup(response_url.text, 'html.parser')

In [155]:
def star_count(stars_str):
    stars_str = stars_str.strip()

    if stars_str[-1] == 'k':
       return int(float(stars_str[:-1]) * 100)
    return int(stars_str)

In [ ]:
#GET ALL INFO ABOUT ALL REPOSITORY

In [216]:
def get_topic_page(topic_urls):
    response = requests.get(topic_urls)
    if response.status_code != 200:
        raise Exception('Failed to get data...'.format(topic_url))

    topic_doc = BeautifulSoup(response.text, 'html.parser')

    return topic_doc

def get_repo_info(h_tags, star_tags):
    a_tags = h_tags.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = 'https://gtihub.com' + a_tags[1]['href']
    sts = star_count(star_tags.text)

    return username, repo_name,sts, repo_url

def get_topic_repo(topic_doc):
    
    h1_class_select = "f3 color-fg-muted text-normal lh-condensed"
    h_tags = topic_doc.find_all('h3', {'class' : h1_class_select})
    
    star_tags = topic_doc.find_all('span',{'id':"repo-stars-counter-star"})
    
    #get all info
    
    topic_repo_dict = {
        'username': [],
        'repo_name':[],
        'stars':[],
        'repo_url':[]
    }

    for i in range( len(h_tags)):
        repo_info  = get_repo_info(h_tags[i], star_tags[i])
    
        topic_repo_dict['username'].append(repo_info[0])
        topic_repo_dict['repo_name'].append(repo_info[1])
        topic_repo_dict['stars'].append(repo_info[2])
        topic_repo_dict['repo_url'].append(repo_info[3])

    return pd.DataFrame(topic_repo_dict)

def scrape_topic(topic_url, topic_name):
    fileName = topic_name + '.csv'
    if os.path.exists(fileName):
        print("The file {} already exists..".format(fileName))
        return
    
    topic_df = get_topic_repo(get_topic_page(topic_url))
    topic_df.to_csv(fileName + '.csv', index= None)


In [212]:
def get_tittle(doc):
        selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
        topic_title_tags= doc.find_all('p',{'class' : selection_class})
        topic_titles = []
    
        for tag in topic_title_tags:
            topic_titles.append(tag.text)
        return topic_titles

def get_desc(doc):
        desc_selection_class = 'f5 color-fg-muted mb-0 mt-1'
        topics_desc_tags= doc.find_all('p',{'class' : desc_selection_class})
        
        topics_desc = []
        
        for tag in topics_desc_tags:
            topics_desc.append(tag.text.strip())
        return topics_desc

def get_topics_urls(doc):
        topic_links_tags = doc.find_all('a', {'class' : "no-underline flex-grow-0"})
        topic_urls = []
        
        for tags in topic_links_tags:
            topic_urls.append('https://github.com/' + tags['href'])
        return topic_urls

def scrape_topics():
    topics_url = "https://github.com/topics"
    respone = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to get data...'.format(topic_url))

    topics_dict = { 
        'title': get_tittle(doc), 
        'description': get_desc(doc),
        'url': get_topics_urls(doc)
    }
    return pd.DataFrame(topics_dict)
    

In [213]:
def scrape_topics_repo():
    print("Scrapeing.. topics list")
    topics_df= scrape_topics()

    for index ,row in topics_df.iterrows():
        print('Scrapping all repositories in the topic "{}"'.format(row['title']))
        scrape_topic(row['url'],row['title'])
    

In [222]:
scrape_topics_repo()

Scrapeing.. topics list
Scrapping all repositories in the topic "Awesome Lists"
Scrapping all repositories in the topic "Chrome"
Scrapping all repositories in the topic "Code quality"
Scrapping all repositories in the topic "Compiler"
Scrapping all repositories in the topic "CSS"
Scrapping all repositories in the topic "Database"
Scrapping all repositories in the topic "Front end"
Scrapping all repositories in the topic "JavaScript"
Scrapping all repositories in the topic "Node.js"
Scrapping all repositories in the topic "npm"
Scrapping all repositories in the topic "Project management"
Scrapping all repositories in the topic "Python"
Scrapping all repositories in the topic "React"
Scrapping all repositories in the topic "React Native"
Scrapping all repositories in the topic "Scala"
Scrapping all repositories in the topic "TypeScript"
